In [1]:
from hyperopt import hp, tpe, fmin
from keras.regularizers import l2
import keras
from Model import RetrainedClassificationModel
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with tf.device("/cpu:0"):
    model = RetrainedClassificationModel('noname')
    model.model.summary()
    model.train(1)

D:\projects\machine learning\Airbus-Ship-Detection-Challenge\Model\RetrainedClassificationModel.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  self.model = Model(input=model.input, output=predictions)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

ResourceExhaustedError: OOM when allocating tensor with shape[100352,50176] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node training/Adam/zeros_26}} = Fill[T=DT_FLOAT, index_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense_1/random_uniform/shape, Adam/decay/initial_value)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adam/zeros_26', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
   

In [5]:
with tf.device("/cpu:0"):
    model = keras.applications.VGG16()
    for layer in model.layers:
        print(layer)
    model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
with tf.device("/cpu:0"):
    model = keras.applications.VGG16(include_top=False, input_shape=(
            224, 224, 3))
    for layer in model.layers:
        print(layer)
    model.summary()

In [ ]:
def trainModel(l2, dropout):

In [ ]:
best = fmin(fn = lambda x: np.poly1d([1, -2, -28, 28, 12, -26, 100])(x),
            space = hp.normal('x', 4.9, 0.5), algo=tpe.suggest, 
            max_evals = 2000)

In [ ]:
import keras

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
I = plt.imread('train.jpg')
I = cv2.resize(I, (224, 224))

In [ ]:
plt.imshow(I)
plt.show()

In [ ]:
with tf.device("/cpu:0"):
    model = keras.applications.vgg16.VGG16()
    x = keras.applications.vgg16.preprocess_input(np.expand_dims(I, 0))
    pred = model.predict(x)

In [ ]:
keras.applications.vgg16.decode_predictions(pred)

In [ ]:
x.mean()